# FunctionGemma Simple Fine-tuning

Fine-tunes FunctionGemma **WITHOUT** system instructions.

**Result:** Model that works with just user message → function call.
No developer turn, no declarations needed at inference.

**Functions:**
- `change_background_color` - Changes app background color
- `change_app_title` - Changes app title
- `show_alert` - Shows alert dialog

**Requirements:**
- A100 GPU runtime (Runtime → Change runtime type → A100)
- HuggingFace account with accepted Gemma license
- HuggingFace token with write access

## 1. Install Dependencies

In [ ]:
# =============================================================================
# Install dependencies (use Colab's pre-installed torch)
# =============================================================================
!pip install -q transformers==4.57.3 datasets accelerate evaluate trl==0.26.2 protobuf sentencepiece
!pip install -q huggingface_hub tensorboard

print("\nDependencies installed!")

## 2. HuggingFace Authentication

In [ ]:
# =============================================================================
# Authenticate with HuggingFace using Colab Secrets
# =============================================================================
from google.colab import userdata
from huggingface_hub import login

HF_TOKEN = userdata.get('HF_TOKEN')

if not HF_TOKEN:
    raise ValueError("HF_TOKEN not found in Colab Secrets. Add it via the key icon.")

login(token=HF_TOKEN)
print("Logged in to HuggingFace!")

## 3. Upload Training Data

**File:** `training_data.jsonl`

**Format per line:**
```json
{"user_content": "make it red", "tool_name": "change_background_color", "tool_arguments": "{\"color\": \"red\"}"}
```

In [ ]:
from google.colab import files
import os

if not os.path.exists('training_data.jsonl'):
    print("Please upload training_data.jsonl:")
    uploaded = files.upload()
else:
    print("training_data.jsonl already exists")

!wc -l training_data.jsonl
!head -1 training_data.jsonl

## 4. Prepare Dataset (SIMPLE - no declarations!)

**Key difference from standard approach:**
- NO developer turn
- NO function declarations
- Just: user message → function call

In [ ]:
import json
from datasets import Dataset

# =============================================================================
# SIMPLE format: user → function call (NO declarations!)
# =============================================================================

def create_conversation(sample):
    """
    Creates SIMPLE training format without system instructions.
    
    Input:
        {"user_content": "make it red", "tool_name": "change_background_color", ...}
    
    Output:
        {
            "messages": [
                {"role": "user", "content": "make it red"},
                {"role": "assistant", "tool_calls": [...]}
            ]
            # NO "tools" key - model learns functions from examples!
        }
    """
    return {
        "messages": [
            {"role": "user", "content": sample["user_content"]},
            {
                "role": "assistant",
                "tool_calls": [
                    {
                        "type": "function",
                        "function": {
                            "name": sample["tool_name"],
                            "arguments": json.loads(sample["tool_arguments"])
                        }
                    }
                ]
            },
        ]
        # NO tools= here! Model learns from examples directly.
    }

# =============================================================================
# Load and convert dataset
# =============================================================================
raw_data = []
with open('training_data.jsonl', 'r') as f:
    for line in f:
        raw_data.append(json.loads(line.strip()))

print(f"Loaded {len(raw_data)} raw examples")

dataset = Dataset.from_list(raw_data)
dataset = dataset.map(create_conversation, remove_columns=dataset.features)

# Split into train/test (90%/10%)
dataset = dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)

print(f"\nDataset prepared:")
print(f"   Train: {len(dataset['train'])} examples")
print(f"   Test:  {len(dataset['test'])} examples")
print(f"\nSample (first example):")
print(json.dumps(dataset['train'][0], indent=2))

## 5. Load Base Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

BASE_MODEL = "google/functiongemma-270m-it"

print(f"Loading {BASE_MODEL}...")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print(f"\nModel loaded!")
print(f"   Parameters: {model.num_parameters():,}")
print(f"   Device: {model.device}")

## 6. Configure Training

In [ ]:
from trl import SFTConfig, SFTTrainer

OUTPUT_DIR = "functiongemma-simple"

training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    
    # Training params
    max_length=512,                     # Shorter - no declarations!
    packing=False,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    
    # Optimizer
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    warmup_ratio=0.1,
    
    # Logging
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    
    # Memory
    bf16=True,
    report_to="tensorboard",
)

print("Training configuration:")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Max length: {training_args.max_length} (shorter - no declarations!)")
print(f"   Learning rate: {training_args.learning_rate}")

## 7. Train!

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

print("Starting training...")
print(f"   Train examples: {len(dataset['train'])}")
print(f"   Eval examples: {len(dataset['test'])}")
print("-" * 50)

train_result = trainer.train()

print("\n" + "=" * 50)
print("Training complete!")
print(f"   Final loss: {train_result.training_loss:.4f}")

## 8. Save Model

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

FINAL_MODEL_DIR = f"{OUTPUT_DIR}-final"
DRIVE_MODEL_DIR = f"/content/drive/MyDrive/{FINAL_MODEL_DIR}"

trainer.save_model(FINAL_MODEL_DIR)
tokenizer.save_pretrained(FINAL_MODEL_DIR)

print(f"Model saved locally to {FINAL_MODEL_DIR}/")

!cp -r {FINAL_MODEL_DIR} /content/drive/MyDrive/

print(f"\nModel copied to Google Drive: {DRIVE_MODEL_DIR}/")
!ls -la {DRIVE_MODEL_DIR}/

## 9. Test Model (SIMPLE - no declarations!)

**Key difference:** Just user message, no developer turn!

In [ ]:
# =============================================================================
# Test SIMPLE format - just user message!
# =============================================================================

test_prompts = [
    "make the background red",
    "rename the app to Hello World",
    "show an alert saying welcome",
    "I want a purple background",
    "set title to My App",
]

print("Testing model (SIMPLE format - no declarations!):")
print("=" * 60)

for prompt in test_prompts:
    # SIMPLE: just user message, NO developer turn, NO tools!
    messages = [
        {"role": "user", "content": prompt}
    ]
    
    input_text = tokenizer.apply_chat_template(
        messages,
        # NO tools= parameter!
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False
    )
    
    response = tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=False
    )
    
    print(f"\nUser: {prompt}")
    print(f"Model: {response.strip()}")
    print("-" * 60)

## 10. Validate Reloaded Model

In [ ]:
import gc

print("Unloading current model...")
del model
del tokenizer
del trainer
gc.collect()
torch.cuda.empty_cache()

print(f"\nReloading model from {FINAL_MODEL_DIR}...")

reloaded_model = AutoModelForCausalLM.from_pretrained(
    FINAL_MODEL_DIR,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    attn_implementation="eager"
)
reloaded_tokenizer = AutoTokenizer.from_pretrained(FINAL_MODEL_DIR)

print(f"Model reloaded on {reloaded_model.device}")

In [ ]:
# =============================================================================
# Test RELOADED model (SIMPLE format)
# =============================================================================
print("Testing RELOADED model:")
print("=" * 60)

reload_test_prompts = [
    "make the background red",
    "rename the app to Hello World",
    "show an alert saying welcome",
]

all_passed = True

for prompt in reload_test_prompts:
    # SIMPLE: just user message!
    messages = [
        {"role": "user", "content": prompt}
    ]

    input_text = reloaded_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = reloaded_tokenizer(input_text, return_tensors="pt").to(reloaded_model.device)

    outputs = reloaded_model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=False,
        pad_token_id=reloaded_tokenizer.pad_token_id
    )

    response = reloaded_tokenizer.decode(
        outputs[0][inputs['input_ids'].shape[1]:],
        skip_special_tokens=False
    )

    is_valid = "<start_function_call>" in response and "<pad>" not in response[:50]
    status = "OK" if is_valid else "FAIL"
    if not is_valid:
        all_passed = False

    print(f"\n[{status}] User: {prompt}")
    print(f"   Model: {response.strip()[:100]}...")
    print("-" * 60)

print("\n" + "=" * 60)
if all_passed:
    print("RELOADED MODEL WORKS!")
    print("Proceed with TFLite conversion.")
else:
    print("RELOADED MODEL BROKEN!")
print("=" * 60)

## 11. Download Model

In [ ]:
!zip -r {FINAL_MODEL_DIR}.zip {FINAL_MODEL_DIR}/

from google.colab import files
files.download(f"{FINAL_MODEL_DIR}.zip")

print(f"\nDownload started: {FINAL_MODEL_DIR}.zip")
print("\nNext steps:")
print("1. Convert to TFLite using functiongemma_to_tflite.ipynb")
print("2. Bundle as .task using functiongemma_tflite_to_task.ipynb")
print("3. Update Flutter chat.dart to use SIMPLE format (no declarations)")